We want to stipulate a polyhedron 
$$|x_i - x_j| \leq d_{i,j}$$

given by 

$$d = (d_{12}, d_{13}, d_{23}) = (1,1,1)$$

We immediately translate this in the system of absolute-valued algebraic inequalities: 

$$\begin{cases}
|x_1 - x_2| \leq 1 \\
|x_1 - x_3| \leq 1 \\
|x_2 - x_3| \leq 1 
\end{cases}$$

Expanding this to remove the complexity of the inequality, we get: 

$$\begin{cases}
x_1 - x_2 \leq 1 \\
x_2 - x_1 \leq 1 \\
x_1 - x_3 \leq 1 \\
x_3 - x_1 \leq 1 \\
x_2 - x_3 \leq 1 \\
x_3 - x_2 \leq 1 
\end{cases}$$

Lastly, in order for our polynomial to be within $\mathbb R^n \setminus \mathbb R (1,1,1)$, we stipulate that 

$$x \perp (1,1,1) \implies x \cdot (1,1,1) = 0 \implies x_1 + x_2 + x_3 = 0$$

Therefore, we have a polynomial defined as the following: 

$$P = \{ x \in \mathbb R^3: Ax \leq b, c^T x = 0\}$$

where 

$$A = \begin{pmatrix} 
1 & -1 & 0 \\
-1 & 1 & 0 \\
1 & 0 & -1 \\
-1 & 0 & 1 \\
0 & 1 & -1 \\
0 & -1 & 1 
\end{pmatrix}$$

$$b = \begin{pmatrix}
1 \\ 
1 \\
1 \\
1 \\
1 \\
1
\end{pmatrix}$$

and 


$$c = \begin{pmatrix}
1 \\ 
1 \\
1 
\end{pmatrix}$$

Now, a strange behavior of Sage is that it wants it's inequalities to be specfied in the following way: 

$$d + Ex \geq 0$$

$$f$$

Therefore, we need 

In [ ]:
a_1 = [1,-1,1,0]
a_2 = [1,1,-1,0]
a_3 = [1,-1,0,1]
a_4 = [1,1,0,-1]
a_5 = [1,0,-1,1]
a_6 = [1,0,1,-1]

In [38]:
P = Polyhedron(ieqs=[a_1,a_2,a_3,a_4,a_5,a_6], eqns=[[0,1,1,1]])

In [39]:
P.Vrepresentation()

(A vertex at (1/3, -2/3, 1/3),
 A vertex at (1/3, 1/3, -2/3),
 A vertex at (2/3, -1/3, -1/3),
 A vertex at (-1/3, 2/3, -1/3),
 A vertex at (-2/3, 1/3, 1/3),
 A vertex at (-1/3, -1/3, 2/3))